## Explaining Gradiet Descent algorithm for prediction using python codes

#### Reading needed Libraries

In [1]:
# code 4.1
import pandas as pd
import numpy as np

### Read needed file and convert variables

In [2]:
## giving referece to file location
file1 = "data_gd.csv"

In [3]:
pd.read_csv

<function pandas.io.parsers.read_csv(filepath_or_buffer:Union[str, pathlib.Path, IO[~AnyStr]], sep=',', delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, skipfooter=0, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, cache_dates=True, iterator=False, chunksize=None, compression='infer', thousands=None, decimal:str='.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, dialect=None, error_bad_lines=True, warn_bad_lines=True, delim_whitespace=False, low_memory=True, memory_map=False, float_precision=None)>

In [4]:
## code 4.2
##read file

df1 = pd.read_csv(file1)

###covert float for numeric calculations
df1["age"] = df1["age"].astype('float')
df1["experience"] = df1["experience"].astype('float')
df1["amount in lakhs"] = df1["amount in lakhs"].astype('float')

### scaling values using z=X-mean/standard deviation

In [5]:
## code 4.3
def scale_vals(dataframe,col1):
    
    mean1 = np.average(dataframe[col1])
    stdev = np.std(dataframe[col1])
    col2 = col1 +"_std"
    dataframe[col2] = (dataframe[col1]-mean1)/stdev
    
    std_vals = {}
    std_vals["mean1"] = mean1
    std_vals["stdev"] = stdev
    
    return std_vals,dataframe


dict1 = {}
dict1["age"],df1= scale_vals(df1,"age")
dict1["experience"],df1= scale_vals(df1,"experience")
dict1["amount in lakhs"],df1 = scale_vals(df1,"amount in lakhs")

In [6]:
print ("age",dict1["age"])
print ("exp",dict1["experience"])
print ("income",dict1["amount in lakhs"])

age {'mean1': 35.0, 'stdev': 10.21763181955584}
exp {'mean1': 8.2, 'stdev': 3.8418745424597094}
income {'mean1': 38.7, 'stdev': 21.04305110957059}


In [7]:
## transformed dataset
df1.head()

,age,experience,amount in lakhs,age_std,experience_std,amount in lakhs_std
0,23.0,3.0,13.0,-1.17444,-1.353506,-1.221306
1,40.0,10.0,75.0,0.48935,0.468521,1.725035
2,30.0,4.0,40.0,-0.48935,-1.093216,0.061778
3,41.0,9.0,45.0,0.58722,0.208232,0.299386
4,47.0,11.0,52.0,1.17444,0.728811,0.632038


### Gradient Descent Algorithm in Python

In [8]:
## code 4.4
###Initialize weights & learning rate
w1 = 1
w2= 5.3 
w3 = 3
initial_weights = [w1,w2,w3]
lr = 0.001

err_lists_all = []
grad_lists_all = []
weight_lists_all = []
pred_lists_all = []

weight_lists_all.append(initial_weights)

###iterate over many iterations
for j in range(0,1000):
    pred_list = []
    error_list = []
        
    grad_age_list = []
    grad_exp_list = []
    grad_bias_list = []

###For each iteration,caluclate error with current weights, 
###calculate gradient and update weights

    for i,row in df1.iterrows():
        age = row["age_std"]
        experience = row["experience_std"]
        act = row["amount in lakhs_std"]
        
        
        ###predict and compute errors
        pred1 = np.round((age * w1) + (experience * w2) + w3,4)
        error = np.power((act-pred1),2)*1/2
        
        pred_list.append(pred1)
        error_list.append(error)
        
        ###calculate gradients
        grad_age = np.round(-(act-pred1) * age,4)
        grad_exp = np.round(-(act-pred1) * experience,4)
        grad_bias = np.round( -(act-pred1),4)
        
        
        grad_age_list.append(grad_age)
        grad_exp_list.append(grad_exp)
        grad_bias_list.append(grad_bias)
    
    ###Calculate sum of gradients
    grad_age_sum = sum(grad_age_list)
    grad_exp_sum = sum(grad_exp_list)
    grad_bias_sum = sum(grad_bias_list)
    
    ###update weights
    w1 = np.round(w1 - lr*grad_age_sum,4)
    w2 = np.round(w1 - lr*grad_exp_sum,4)
    w3 = np.round(w1 - lr*grad_bias_sum,4)
    
    ###log errors, gradients, weights and predictions
    err_sum = sum(error_list)
    err_lists_all.append(err_sum)
    grad_lists_all.append([grad_age_sum,grad_exp_sum,grad_bias_sum])
    weight_lists_all.append([w1,w2,w3])
    pred_lists_all.append(pred_list)
    

        

In [9]:
## code 4.5
###create ml dataset
ml_dataset = pd.DataFrame(err_lists_all,columns=['errors'])
ml_dataset["gradients"] = grad_lists_all
ml_dataset["weights"] = weight_lists_all[:-1]


In [10]:
ml_dataset["ind"] = ml_dataset.index

In [11]:
ml_dataset.head()

,errors,gradients,weights,ind
0,209.698533,"[50.708, 56.28690000000001, 29.9999]","[1, 5.3, 3]",0
1,17.267911,"[13.714300000000001, 11.7973, 9.193]","[0.9493, 0.893, 0.9193]",1
2,17.511327,"[13.832499999999998, 11.9918, 9.264200000000002]","[0.9356, 0.9238, 0.9264]",2
3,17.027786,"[13.578399999999998, 11.7374, 9.1249]","[0.9218, 0.9098, 0.9125]",3
4,16.568952,"[13.332299999999998, 11.4919, 8.991000000000001]","[0.9082, 0.8965, 0.8991]",4


### Getting the weights & intercept from the last iteration

In [12]:
ml_dataset[ml_dataset.ind==99]

,errors,gradients,weights,ind
99,4.648743,"[2.3452999999999995, 0.5241, 2.9909999999999997]","[0.3021, 0.3015, 0.2991]",99


### Our equation becomes what is shown below

### income_pred = age \* 0.3021 + experience \* 0.3015 + 0.2991
##### here age and experience are standardized values

### Predicting for age 36 and experience 12

### Standardizing Values and Substitue in above eqaution to get predicted income

In [13]:
### Code 4.6
age_conv = (36 - dict1["age"]["mean1"])/dict1["age"]["stdev"]
exp_conv = (12- dict1["experience"]["mean1"])/dict1["experience"]["stdev"]

In [14]:
age_conv,exp_conv

(0.09787003658578391, 0.9891004919611719)

In [15]:
### Code 4.7
std_amount = age_conv*0.3021 + exp_conv*0.3015 + 0.2991

In [16]:
std_amount

0.6268803363788586

In [17]:
##get final amount as final income for standardized age and experience 
### code 4.8
final_amount = (std_amount*dict1["amount in lakhs"]["stdev"]) + dict1["amount in lakhs"]["mean1"]

In [18]:
final_amount

51.89147495800513